In [1]:
import json
import jiwer
from tqdm import tqdm

# Prepare Data

In [16]:
import jiwer
from tqdm import tqdm

data_name = 'aishell'
if (data_name in ['csj']):
    recog_set = ['train','dev', 'eval1', 'eval2', 'eval3']
elif (data_name in ['aishell2']):
    recog_set = [ 'train'] #'dev_ios', 'test_mic', 'test_ios', 'test_android',
elif (data_name in ['librispeech']):
    recog_set = ['dev_clean', 'dev_other', 'test_clean', 'test_other']
elif (data_name in ['aishell']):
    recog_set = ['train'] #,'dev', 'test'
elif (data_name in ['tedlium2']):
    recog_set = ['train','dev', 'dev_trim' ,'test']
setting = ['withLM', ' noLM']
nbest = 50
check_dataset = True

In [5]:
data_name

'tedlium2'

In [6]:
recog_set

['train', 'dev', 'dev_trim', 'test']

In [31]:
data_name = 'tedlium2'
if (data_name in ['csj']):
    recog_set = ['train','dev', 'eval1', 'eval2', 'eval3']
elif (data_name in ['aishell2']):
    recog_set = ['train', 'dev_ios', 'test_mic', 'test_ios', 'test_android']
elif (data_name in ['librispeech']):
    recog_set = ['train', 'dev_clean', 'dev_other', 'test_clean', 'test_other']
elif (data_name in ['aishell']):
    recog_set = ['train', 'dev', 'test'] #,'dev', 'test'
elif (data_name in ['tedlium2']):
    recog_set = ['train','dev', 'dev_trim' ,'test']
setting = ['withLM', 'noLM']
FLAG = True
print(f'recog set:{type(recog_set)}')
for recog in recog_set:
    print(f'recog:{recog}')
    for s  in setting:
        with open(f"/mnt/disk6/Alfred/Rescoring/data/{data_name}/data/{s}/{recog}/data.json") as f :
            data_json = json.load(f)
            if isinstance(data_json, dict):
                print(f"/mnt/disk6/Alfred/Rescoring/data/{data_name}/data/{s}/{recog}/data.json is dict")
                FLAG = False
            count = 0
            for data in data_json:
                if (len(data['hyps']) == 1):
                    print(data['name'])
                    count += 1
            
            print(f"{data_name} - {recog} - {s}: OnlyOneDataCount:{count}")
    
if (FLAG):
    print(f'{data_name}: All datas are list')

recog set:<class 'list'>
recog:train
tedlium2 - train - withLM: OnlyOneDataCount:0
tedlium2 - train - noLM: OnlyOneDataCount:0
recog:dev
tedlium2 - dev - withLM: OnlyOneDataCount:0
tedlium2 - dev - noLM: OnlyOneDataCount:0
recog:dev_trim
tedlium2 - dev_trim - withLM: OnlyOneDataCount:0
tedlium2 - dev_trim - noLM: OnlyOneDataCount:0
recog:test
tedlium2 - test - withLM: OnlyOneDataCount:0
tedlium2 - test - noLM: OnlyOneDataCount:0
tedlium2: All datas are list


In [ ]:
# file_type = '.txt'
# trn_name = '.trn'
# if (data_name in ['tedlium2', 'librispeech'] and check_dataset):
#     file_type = ".wrd.txt"
#     trn_name = '.wrd.trn'
# print(file_type)

for s in setting:
    for task in recog_set:
        nbest_err = []
        nbest_num = []

        data_list = []
        print(f'{s}:{task}')
        print(f'../data/{data_name}/raw/{s}/{task}/data.json')
        with open(f'../data/{data_name}/raw/{s}/{task}/data.json', 'r') as f:
        # with open(f'/mnt/disk4/Alfred/espnet/egs/aishell/asr1/exp/rescoring/train_pytorch_rescoring_withLM/decode_train_decode_lm_4/data.json', 'r') as f:
            json_data = json.load(f)
            for k in json_data['utts'].keys():
                nbest_num.append(len(json_data['utts'][k]['output']))
            print(f'len of best_num:{len(nbest_num)}')
        
        for k in tqdm(json_data['utts'].keys(), ncols= 100):
            hyps = []
            errs = []
            scores = []
            if ('am_score' in json_data['utts'][k]['output'][0].keys()):
                am_scores = []
            else:
                print(f'no am_score:{k}')
                am_scores = None
            if ('ctc_score' in json_data['utts'][k]['output'][0].keys()):
                ctc_scores = []
            else:
                print(f'no ctc_score:{k}')
                ctc_scores = None
            
            if (data_name in ['librispeech', 'tedlium2']):
                ref = json_data['utts'][k]['output'][0]["text"]
            else:
                ref = json_data['utts'][k]['output'][0]["token"]
            
            if ('lm_score' in json_data['utts'][k]['output'][0].keys()):
                lm_scores = []
            else:
                # print(f'no lm_score:{k}')
                lm_scores = None
            for hyp in (json_data['utts'][k]['output']):
                if (data_name in ['librispeech', 'tedlium2']):
                    if ("<eos>" in hyp['rec_token'] ):
                        hyp_token = hyp['rec_token'][:-5]
                    else:
                        hyp_token = hyp['rec_token'][:]
                    hyp_token = hyp_token.replace(" ", "")
                    hyp_token = hyp_token.replace("▁", " ")
                else:
                    hyp_token = hyp['rec_token'].split() # remove <eos>
                    if ('<eos>' in hyp_token):
                        hyp_token = hyp_token[:-1]
                    hyp_token = " ".join(hyp_token)
                hyps.append(hyp_token.lower())
                
                if (am_scores is not None):
                    am_scores.append(hyp['am_score'])
                if (ctc_scores is not None):
                    ctc_scores.append(hyp['ctc_score'])
                if (lm_scores is not None):
                    lm_scores.append(hyp['lm_score'])
                
                scores.append(hyp['score'])
                
                measures = jiwer.compute_measures(ref, hyp_token)

                errs.append(
                    {
                        'err': measures['wer'],
                        'hit': measures['hits'],
                        'sub': measures['substitutions'],
                        'del': measures['deletions'],
                        'ins': measures['insertions'],
                    }
                )

            if (len(hyps) <= 1):
                print(f'name:{k} \n hyp:{hyps}')
            data_list.append(
                {
                    'name': k,
                    'hyps': hyps,
                    'am_score': am_scores,
                    'ctc_score': ctc_scores,
                    "lm_score": None if lm_scores is None else lm_scores,
                    "score": scores,
                    'err': errs,
                    'ref': ref.lower()
                }
            )

        with open(f"../data/{data_name}/data/{s}/{task}/data.json", 'w') as f:
            json.dump(data_list, f, ensure_ascii=False, indent = 1)

# Combine librispeech 1~9

In [ ]:
from tqdm import tqdm

In [ ]:
data_list = []
nbest_err = []
nbest_num = []

data_name = "librispeech"

low_nBest = 0
for num in range(1, 9):
    with open(f"/mnt/disk4/Alfred/espnet/egs/librispeech/asr1/exp/train_960_pytorch_rescoring_withLM/decode_train_960_{num}_model.val5.avg.best_decode_pytorch_transformer_lm/data.json") as f:
        json_data = json.load(f)

        for k in json_data['utts'].keys():
            if (len(json_data['utts'][k]['output']) < 50):
                low_nBest += 1
            nbest_num.append(len(json_data['utts'][k]['output']))
        print(f'len of data:{len(nbest_num)}')

        for k in tqdm(json_data['utts'].keys(), ncols= 100):
            hyps = []
            errs = []
            scores = []
            if ('am_score' in json_data['utts'][k]['output'][0].keys()):
                am_scores = []
            else:
                am_scores = None
            if ('ctc_score' in json_data['utts'][k]['output'][0].keys()):
                ctc_scores = []
            else:
                ctc_scores = None
            
            if (data_name in ['librispeech', 'tedlium2']):
                ref = json_data['utts'][k]['output'][0]["text"]
            else:
                ref = json_data['utts'][k]['output'][0]["token"]
            
            if ('lm_score' in json_data['utts'][k]['output'][0].keys()):
                lm_scores = []
            else:
                lm_scores = None
            for hyp in (json_data['utts'][k]['output']):
                if (data_name in ['librispeech', 'tedlium2']):
                    if ("<eos>" in hyp['rec_token'] ):
                        hyp_token = hyp['rec_token'][1:-5]
                    else:
                        hyp_token = hyp['rec_token'][1:]
                    hyp_token = hyp_token.replace(" ", "")
                    hyp_token = hyp_token.replace("▁", " ")
                else:
                    hyp_token = hyp['rec_token'].split() # remove <eos>
                    if ('<eos>' in hyp_token):
                        hyp_token = hyp_token[:-1]
                    hyp_token = " ".join(hyp_token)
                hyps.append(hyp_token.lower())
                
                if (am_scores is not None):
                    am_scores.append(hyp['am_score'])
                if (ctc_scores is not None):
                    ctc_scores.append(hyp['ctc_score'])
                if (lm_scores is not None):
                    lm_scores.append(hyp['lm_score'])
                
                scores.append(hyp['score'])
                
                measures = jiwer.compute_measures(ref, hyp_token)

                errs.append(
                    {
                        'err': measures['wer'],
                        'hit': measures['hits'],
                        'sub': measures['substitutions'],
                        'del': measures['deletions'],
                        'ins': measures['insertions'],
                    }
                )
            data_list.append(
                {
                    'name': k,
                    'hyps': hyps,
                    'am_score': am_scores,
                    'ctc_score': ctc_scores,
                    "lm_score": None if lm_scores is None else lm_scores,
                    "score": scores,
                    'err': errs,
                    'ref': ref.lower()
                }
            )

with open(f"../data/{data_name}/data/withLM/train/data.json", 'w') as f:
    json.dump(data_list, f, ensure_ascii=False, indent =1)

print(f"low_nBest:{low_nBest}")

In [ ]:
for setting in ['withLM']:
    with open(f'/mnt/disk6/Alfred/Rescoring/data/librispeech/data/{setting}/dev_clean/data.json') as clean, \
         open(f'/mnt/disk6/Alfred/Rescoring/data/librispeech/data/{setting}/dev_other/data.json') as other:

        clean_json = json.load(clean)
        other_json = json.load(other)

        # print(f'{type(clean_json)}')
        # break

        concat_json = clean_json + other_json

        with open(f'/mnt/disk6/Alfred/Rescoring/data/librispeech/data/{setting}/valid/data.json', 'w') as f:
            json.dump(concat_json, f, ensure_ascii=False, indent=2)

## Result trn

In [ ]:
dataname = 'aishell'
name = "Audio_Aware"
setting = 'noLM'
recog_set = ['dev', 'test']
nbest = 1
best_type = 'bart' 
training = 'MD'

# BART

In [ ]:
for r in recog_set:
    json_name = f"./src/Correction/data/{dataname}/{setting}/{r}/{nbest}{best_type}/correct_data.json"
    with open(json_name, 'r') as f, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/{nbest}{best_type}/hyp.trn', 'w') as hyp, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/{nbest}{best_type}/ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j.keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j[k]['hyp']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j[k]['ref']
            ref.write( f'{ref_seq} ({k})\n' )

# RescoreBert

In [ ]:
for task in recog_set:
    json_name = f"./data/{dataname}/{task}/{training}/{setting}/{nbest}best_rescore_data.json"
    with open(json_name, 'r') as f, \
        open(f'./data/{dataname}/{task}/{training}/{setting}/{nbest}best_hyp.trn', 'w') as hyp, \
        open(f'./data/{dataname}/{task}/{training}/{setting}/{nbest}best_ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['text_token']
            ref.write( f'{ref_seq} ({k})\n' )
    print(f'sclite -r ./{nbest}best_hyp.trn -h  ./{nbest}best_ref.trn -i rm -o all stdout >  ./{nbest}best_result.txt')

In [ ]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/{name}/rescore_data.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/{name}/hyp_rescore.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/{name}/ref_rescore.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['rec_text']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['ref_text']
            ref.write( f'{ref_seq} ({k})\n' )

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
import json
import torch
# for r in recog_set:
best_weight = 0
best_cer = 100
with open(f"./data/{dataname}/dev/{nbest}best_{training}_rescore_data_{setting}.json") as f:
    data = json.load(f)
    
    for n in range(101):
        weight = n * 0.01
        c = 0
        s = 0
        de = 0
        i = 0
        for d in data:
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            c += cer[max_index][0]
            s += cer[max_index][1]
            de += cer[max_index][2]
            i += cer[max_index][3]
 
        cer = (s + de + i) / (c + s + de)

        if (cer < best_cer):
            best_cer = cer
            best_weight = weight

print(f'best weight:{best_weight} with cer:{best_cer}')

for task in recog_set:
    with open(f"./data/{dataname}/{task}/{nbest}best_{training}_rescore_data_{setting}.json", 'r') as f, \
         open(f'./data/{dataname}/{task}/hyp_mlm.trn', 'w') as h,\
         open(f'./data/{dataname}/{task}/ref_mlm.trn', 'w') as g:
        data = json.load(f)
        for n, d in enumerate(data):
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
            ref = d['ref'][5:-5]
            token = d['token']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            best_hyp = token[max_index]
            sep = best_hyp.index(102)
            hyp_seq = tokenizer.convert_ids_to_tokens(best_hyp[1:sep])

            h.write( f'{" ".join(hyp_seq)} ({r}_{n})\n' )
            g.write( f'{" ".join(list(ref))} ({r}_{n})\n')
